In [2]:
%load_ext autoreload
%autoreload 2
!pwd
!hostname

/home/ajasja/projects/truncator
dig1


In [4]:
import numpy as np
from glob import glob
import numpy as np
import os
import truncator
import pandas as pd

In [9]:
import dask
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='interactive',
                     walltime="12:00:00")
#cluster.scale(40)
cluster.adapt(minimum=0, maximum=40, interval='5s')
print(cluster.scheduler_address); print(cluster.dashboard_link)
from dask.distributed import Client, progress
client = Client(cluster); client

tcp://172.16.131.31:4901
http://172.16.131.31:8787/status


Client Scheduler: tcp://172.16.131.31:4901 Dashboard: http://172.16.131.31:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [10]:
structs = glob("/home/fseeger/native_PPIs/Zdock_ds/relaxed_zdock/*.pdb")
structs;

In [12]:
%%time
sf= []
for pdb in structs:
    sf.append(client.submit(truncator.score_interface,pdb,
                        script_name="truncator/xml/05_specific_interface_markers.xml",
                        out_dir="out/native", test_run=False, skip_existing=True)
             )
sf

CPU times: user 50.5 ms, sys: 23.6 ms, total: 74.1 ms
Wall time: 69.8 ms


In [13]:
dask.distributed.wait(sf, timeout=5)

TimeoutError: Timeout

In [ ]:
retry_futures = []
for k, v in client.who_has().items():
    future = dask.distributed.Future(key=k, client=client, inform=True, state=None)
    if future.status == "error":
        retry_futures.append(future)

In [ ]:
cluster.close()

In [ ]:
import truncator
score_files = sorted(glob("testing_out/grouped/*_IA.sc"))
score_files

In [ ]:
scores = truncator.read_score_files(score_files, skiprows=False, load_seq=False).set_index('decoy')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
score

In [ ]:
files = sorted(glob("testing_out/grouped/*_IA*.sc"), reverse=True)
files

In [ ]:
scores_sasa = [col for col in scores.columns if "sasa" in col.lower()]
scores_sasa
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_sasa].transpose())

In [ ]:
scores_hbond = [col for col in scores.columns if "loop_sc" in col.lower()]
scores_hbond
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_hbond].transpose())

In [ ]:
truncator.score_interface("data/ZCON_37_N3_C1_2.pdb",
                script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                out_dir="testing_out/grouped", tee=False, test_run=True)

In [ ]:

scores = truncator.read_score_files(['testing_out/grouped/ZCON_37_N3_C1_2_IA.sc'], skiprows=False, load_seq=False).set_index('description')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
interesting_cols = """
interface_averageDegree_chA
interface_averageDegree_chB
interface_buried_sasa
interface_contactcount
interface_fxn_hydrophobic
interface_hbonds
interface_holes
interface_hydrophobic_sasa
interface_polar_sasa
interface_salt_bridges
interface_sc
interface_sc_int_area
interface_unsat_hbond1
interface_unsat_hbond2
cross_interface_energy_IA
cross_interface_hbonds_IA
delta_unsat_hbonds_IA
hbond_energy_IA
interface_delta_sasa_IA
interface_hydrophobic_sasa_IA
interface_polar_sasa_IA
exposed_hydrophobics
exposed_polars
exposed_total
""".split()
#print("\n".join(list(df.columns)))
df[interesting_cols].transpose()

In [ ]:
import os
import subprocess, sys
os.system("echo TEST")
!ls -l

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.max_columns=None
df.transpose()

In [ ]:
import dask
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='interactive',)
cluster.scale(4)

from dask.distributed import Client
client = Client(cluster)